In [2]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated
from langchain_groq import ChatGroq


project_root = Path().resolve().parent  
sys.path.append(str(project_root))

load_dotenv()

from src.receipt_intelligence.config import settings


class ConversationState(TypedDict):
    """
    State class for the LangGraph workflow. 
    It keeps track of the information necessary to maintain a coherent conversation between the user and the bot.

    Attributes:
        messages: A list of messages between the user and the bot.
        message_type (str): Current message type, used for routing the message to the correct node.
        summary (str): A summary of the conversation. This is used to reduce the token usage of the model.
    """

    messages: Annotated[list, add_messages]
    message_type: str | None
    summary: str



In [25]:
testState = ConversationState(messages=[], message_type=None, summary="")

test_message = {"role": "human", "content": "Hello, bot!"}


testState["messages"] = testState.get("messages") + [test_message]


In [39]:
last_message = testState["messages"][-1]
print(last_message['content'])

Hello, bot!


In [5]:
def get_chat_model(temperature: float = 0.7, model_name: str = settings.GROQ_LLM_MODEL) -> ChatGroq:
    """
    Returns a ChatGroq model instance with the given temperature and model name.
    Defaults to standard settings.
    """
    return ChatGroq(
        api_key=os.getenv("GROQ_API_KEY"),
        model_name=model_name,
        temperature=temperature,
    )

In [46]:
def classify_message(state: ConversationState):
    
    last_message = state["messages"][-1]
    model = get_chat_model()

    prompt = [
            {
                "role": "system",
                "content": "Classify the user message as either: "
                "'generate_sqlquery_node': if the user asks a question directly related to receipt data"
                "'introduction_node': if the user asks about anything unrelated to the receipt data"
                "You can only output one of these: 'generate_sqlquery_node' or 'introduction_node'"
            },
            {
                "role": "user",
                "content": last_message['content']
            }
        ]

    response = model.invoke(prompt)

    return response.content

In [48]:
result = classify_message(testState)

In [50]:
type(result)
print(result)

introduction_node


In [ ]:

from src.receipt_intelligence.modules.conversation_service.state import ConversationState
from src.receipt_intelligence.modules.conversation_service.graph import graph   


user_input = input("Message: ")

# Initial state

state: ConversationState = {
    "messages": [],
    "message_type": None,
    "summary": ""
}

state["messages"].append(HumanMessage(content="Hello there!"))

last_message = state["messages"][-1]


KeyError: 'messages'

In [8]:
print(last_message.content)

AttributeError: 'dict' object has no attribute 'content'